In [30]:
using GamsStructure
using CSV

In [31]:
set_dir = "base_data_2017/set"
parm_dir = "base_data_2017/parameter"

GU = GamsUniverse()


@GamsSets(GU,set_dir,begin
    :sr, "Super Regions in WiNDC Database"
    :r, "Regions in WiNDC Database"
    #:yr, "Years in WiNDC Database"
    :i, "BEA Goods and sectors categories"
end)

alias(GU,:i,:s)

@GamsDomainSets(GU,parm_dir,begin
    :n, :usatrd_units, 2,"Dynamically created set from parameter usatrd, NAICS codes"
    :yr, :usatrd_units,   3, "Dynamically created set from parameter usatrd_units, Years in USA trade data set (2002-2016)"
    :t, :usatrd_units, 4, "Dynamically create set from parameter usatrd, Trade type (import/export)"
    :usa_units_domain, :usatrd_units, 5, "Dynamically create set from parameter usatrd, Units"
end)

@GamsParameters(GU,parm_dir,begin
    :usatrd_units, (:sr,:n,:yr,:t,:usa_units_domain), "Trade data with units as domain", [1,2,3,4,5]
end)

@GamsParameters(GU,begin
    :usatrd,    (:yr,:r,:i,:t), "Mapped trade data" #"Trade data without units"
    #:usatrd_,   (:yr,:r,:i,:t), "Mapped trade data"
    :usatrd_shr,(:yr,:r,:i,:t), "Share of total trade by region"
    :shrchk,    (:s,:t), "Comparison between imports and exports"
end)

1;

In [32]:
function load_usatrd_map(;map_file ="maps/mapusatrd.csv" )

    mappce = Dict()
    F = CSV.File(map_file,stringtype=String,
                silencewarnings=true,header = [:a,:b,:c])

    for row in F
        val = Symbol(row[1])
        key = Symbol(string(row[2]))
        if val ∈ keys(mappce)
            push!(mappce[val],key)
        else
            mappce[val] = [key]
        end
    end
    return mappce
end

usatrd_map = load_usatrd_map();

In [46]:
"Scale originally millions. Make billions"
for i∈keys(usatrd_map)
    if length(usatrd_map[i])==1
        GU[:usatrd][:yr,:r,[i],:t] = permutedims(GU[:usatrd_units][:r,usatrd_map[i],:yr,:t,:],(2,1,3))*1e-3
    else
        GU[:usatrd][:yr,:r,[i],:t] = permutedims(sum(GU[:usatrd_units][:r,usatrd_map[i],:yr,:t,:],dims=2),(3,1,2,4))*1e-3
    end
end

In [40]:
sum(GU[:usatrd_units][:r,usatrd_map[:pri],:yr,:t,:],dims=2)

51×1×2 Array{Float64, 3}:
[:, :, 1] =
  481.655625
 3265.206346
 2971.904871
 2479.9923279999994
 3665.5237700000002
 5341.861161000001
  150.043202
    2.8451869999999997
  262.63042699999994
  677.386696
    ⋮
  258.032477
   51.706961
 9077.678147999999
  402.45441500000004
   19.069933999999996
  140.28420199999996
 5266.460017
    4.521772
  804.4417990000001

[:, :, 2] =
  146.08419199999997
 1453.951574
 1075.051151
 1169.320544
 1654.3994429999998
  973.3338419999999
  219.02343699999997
   13.575931999999998
   32.858869
  239.73001299999999
    ⋮
  226.87699499999997
   26.669938
 3310.798842
  247.853931
   18.960461
  117.18408
 2339.376111
   14.165572
  300.23561199999995

In [20]:
length(GU[:usa_units_domain])

1